# Showing that Norton-Thevenin Can Equal CLA

## Using the `frclim.ntfl` routine to compute coupled system response

This and other jupyter notebooks are available here: https://github.com/twmacro/pyyeti/tree/master/docs/tutorials.

The goal of this tutorial is to show that the Norton-Thevenin (NT) method of coupling a "Source" model and a "Load" model together and computing the system response can be equivalent to coupling the models together the "old fashioned" way and computing the system response. The old fashioned way is what is done in a standard Nastran superelement run; that is, it forms coupled system mass, damping, and stiffness matrices by enforcing boundary compatibility, computes system modes (typically), and solves the equations of motion. A typical Coupled Loads Analysis (CLA) couples models together in this fashion.

The NT approach also enforces boundary compatibility and computes coupled system response, but does not form system matrices or compute system modes. Instead, it uses the "apparent masses" of the Source and the Load and the "free acceleration" of the Source to compute the system level responses. The NT coupling equations and their derivation are shown here: [pyyeti.frclim.ntfl](../modules/generated/pyyeti.frclim.ntfl.html#pyyeti.frclim.ntfl) (see equations 4 & 5). Note that the NT method is naturally a frequency domain method and, for simplicity, this tutorial will compute only frequency domain responses. Also, these models have a **statically-indeterminate** interface, showing that the method is general.

To demonstrate that the NT approach can equal CLA approach for a frequency domain analysis, we'll pick two models already available in the pyYeti package. The "inboard" model will be the Source, and the "outboard" model will be the Load. These happen to both be Hurty-Craig-Bampton (HCB) models, but that is not necessary for applying the NT method.

*Note:*

>No matter which format the models are in, be sure to include the complete boundary flexibility in both the Source and Load models when applying the NT method. HCB models will normally contain the required boundary flexibility in the normal and constraint modes. While that is likey the case with our models, for the demonstration here, we will simply considered them to be "exact" and will make no effort to ensure they accurately represent the original FEM models.

Steps:
1. Load models
2. Set up a frequency domain forcing function
3. Couple the models together the old fashioned way
   1. Compute system responses the old fashioned way
4. Couple the models together via the NT approach
   1. Compute apparent masses and free acceleration
   2. Compute system responses via the NT method
5. Compare results



For this example, we'll use a simple space-station model that has been reduced into two separate Hurty-Craig-Bampton models: an "inboard" model and an "outboard" model. These model attach at nodes 3, 11, 19, and 27.

The inboard model (the Source):

.. image:: inboard.png
<img src="./inboard.png" />

The outboard model (the Load):

.. image:: outboard.png
<img src="./outboard.png" />

---
First, do some imports:

In [ ]:
import inspect
from pathlib import Path
import numpy as np
from scipy import linalg as la
import matplotlib.pyplot as plt
from pyyeti import ode, frclim
from pyyeti.nastran import bulk, op4, n2p
from pyyeti.pp import PP

Some settings specifically for the jupyter notebook:

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [6.4, 4.8]
plt.rcParams['figure.dpi'] = 150.

Need path to data files:

In [ ]:
srcdir = Path(inspect.getfile(frclim)).parent / "tests" / "nas2cam_extseout"

#### Load model data

Also add 2% critical modal damping for both models:

In [ ]:
zeta = 0.02
ids = ("out", "in")
uset, cords, b = {}, {}, {}
mats = {}
for id in ids:
    uset[id], cords[id], b[id] = bulk.asm2uset(srcdir / f"{id}board.asm")
    mats[id] = op4.read(srcdir / f"{id}board.op4")

    # add damping:
    bxx = 0 * mats[id]["kxx"]
    q = ~b[id]
    lam = np.diag(mats[id]["kxx"])[q]
    damp = 2 * np.sqrt(lam) * zeta
    bxx[q, q] = damp
    mats[id]["bxx"] = bxx
    
PP(mats, 2);

For convenience and ease of reading, create some shorter names for the mass, damping, and stiffness matrices:

In [ ]:
maa = {
    "in": mats["in"]["mxx"],
    "out": mats["out"]["mxx"],
}
kaa = {
    "in": mats["in"]["kxx"],
    "out": mats["out"]["kxx"],
}
baa = {
    "in": mats["in"]["bxx"],
    "out": mats["out"]["bxx"],
}

#### Define the forcing function

We'll use the first TLOAD vector of "inboard" to apply a force to the coupled system. This load vector applies forces to nodes 13 & 21. For simplicity, we'll keep the forces constant across the frequency range.

The analysis will cover the frequency range 0.01 to 100.0 Hz on a logarithmic scale.

In [ ]:
freq = np.geomspace(0.01, 100.0, 4 * 167 + 1)

# use first TLOAD vector and expand it:
force_in = mats["in"]["px"][:, :1] @ np.ones((1, len(freq)))
force_in.shape

Take a quick look at the inboard applied forces:

In [ ]:
np.set_printoptions(precision=3, linewidth=130, suppress=True)
force_in[:, :5]

#### Couple models together the old fashioned way:

We'll define a matrix 'S' that will enforce boundary compatibility. It defines the relationship from the DOF chosen to be the independent set ($p_{ind}$) to all the DOF ($p_{all}$):

$$p_{all} = \mathbf{S} \cdot p_{ind}$$

$$\left \{ \begin{array}{c} in_b \\ in_q \\ out_b \\ out_q \end{array} \right \}
= \mathbf{S} \left \{ \begin{array}{c} in_b \\ in_q \\ out_q \end{array} \right \}$$

Since $out_b = in_b$:

$$\left \{ \begin{array}{c} in_b \\ in_q \\ out_b \\ out_q \end{array} \right \}
= \left [ \begin{array}{ccc}
I & 0 & 0 \\
0 & I & 0 \\
I & 0 & 0 \\
0 & 0 & I \\
\end{array} \right ]
\left \{ \begin{array}{c} in_b \\ in_q \\ out_q \end{array} \right \}$$

The following code takes advantage of the fact that the b-set is ordered first for both models.

In [ ]:
m = maa["in"].shape[0]
n = maa["out"].shape[0]
nb = np.count_nonzero(b["in"])
S = {}
S["in"] = np.block(
    [
        [np.eye(nb), np.zeros((nb, m + n - 2 * nb))],
        [np.zeros((m - nb, nb)), np.eye(m - nb), np.zeros((m - nb, n - nb))],
    ]
)
S["out"] = np.block(
    [
        [np.eye(nb), np.zeros((nb, m + n - 2 * nb))],
        [np.zeros((n - nb, m)), np.eye(n - nb)],
    ]
)
S["tot"] = np.vstack((S["in"], S["out"]))

Apply the S matrix to couple the models together:

In [ ]:
mc = S["tot"].T @ la.block_diag(maa["in"], maa["out"]) @ S["tot"]
kc = S["tot"].T @ la.block_diag(kaa["in"], kaa["out"]) @ S["tot"]
bc = S["tot"].T @ la.block_diag(baa["in"], baa["out"]) @ S["tot"]

As a check, compare coupled model frequencies to Nastran:

In [ ]:
lam, phi = la.eigh(kc, mc)
freqsys = np.sqrt(abs(lam)) / 2 / np.pi
eigen = bulk.rdeigen(srcdir / "assemble.out")
np.allclose(freqsys[6:], eigen[0]["cycles"][6:])

##### Solve the equations of motion the old fashioned way

We'll solve the equations of motion using [pyyeti.ode.FreqDirect](../modules/generated/pyyeti.ode.FreqDirect.html#pyyeti.ode.FreqDirect) and recover responses for both components. Note that upstream data recovery could be done easily with the "mug1", "mef1" and "mes1" matrices loaded from the op4 files above.

Note: normally, [pyyeti.ode.SolveUnc](../modules/generated/pyyeti.ode.SolveUnc.html#pyyeti.ode.SolveUnc) is preferred, but that solver did not work for this problem due to numerical issues.

In [ ]:
fs = ode.FreqDirect(mc, bc, kc)
force = S["in"].T @ force_in
sol = fs.fsolve(force, freq)

# recover displacements, velocities, and accelerations for both components:
d, v, a = {}, {}, {}
ifforce = {}
for id in ids:
    d[id] = S[id] @ sol.d
    v[id] = S[id] @ sol.v
    a[id] = S[id] @ sol.a

    ifforce[id] = (maa[id] @ a[id] + baa[id] @ v[id] + kaa[id] @ d[id])[:nb]

Do some sanity checks on the results. The boundary accelerations, velocities, and displacements should match, and the boundary forces from one component on the other should be equal and opposite.

*Note:*

>In the lowest frequencies, we lose some numerical precision on the boundary forces. We'll explore this a little more later in the NT context. For now, we'll note that the forces are still very close and that above 1.0 Hz, the `np.allclose` check passes.

In [ ]:
print(f'{np.allclose(a["in"][:nb], a["out"][:nb]) = }')
print(f'{np.allclose(v["in"][:nb], v["out"][:nb]) = }')
print(f'{np.allclose(d["in"][:nb], d["out"][:nb]) = }')

print()
print(f'{np.allclose(-ifforce["out"], ifforce["in"] - force[:nb]) = }')
print(f'{abs(ifforce["out"] + (ifforce["in"] - force[:nb])).max() = }')

pv = freq >= 1.0
print()
print(f'For freq >= 1.0:\n\t{np.allclose(-ifforce["out"][:, pv], ifforce["in"][:, pv] - force[:nb, pv]) = }')

#### Use the NT approach to couple models together

First, we'll compute the "free acceleration" $A_s$:

In [ ]:
fs_in = ode.FreqDirect(maa["in"], baa["in"], kaa["in"])
As = fs_in.fsolve(force_in, freq).a[:nb]

Next, we'll compute the apparent masses and call [pyyeti.frclim.ntfl](../modules/generated/pyyeti.frclim.ntfl.html#pyyeti.frclim.ntfl) to compute coupled system responses.

For illustration and as a check on the routine, we'll use both methods within [pyyeti.frclim.calcAM](../modules/generated/pyyeti.frclim.calcAM.html#pyyeti.frclim.calcAM) to compute the apparent masses. Those two methods are labeled as `cbtf` and `fsolve` here; for more information, see [pyyeti.frclim.calcAM](../modules/generated/pyyeti.frclim.calcAM.html#pyyeti.frclim.calcAM).

Note that we get a warning from [pyyeti.frclim.calcAM](../modules/generated/pyyeti.frclim.calcAM.html#pyyeti.frclim.calcAM) about switching to the [pyyeti.ode.FreqDirect](../modules/generated/pyyeti.ode.FreqDirect.html#pyyeti.ode.FreqDirect) solver. This is due to the same issues as noted above. We could avoid the warning by preparing our own `fs` input to [pyyeti.frclim.calcAM](../modules/generated/pyyeti.frclim.calcAM.html#pyyeti.frclim.calcAM).

In [ ]:
AM = {}
NT = {}
for method in ("cbtf", "fsolve"):
    AM[method] = {}
    for id in ids:
        if method == "cbtf":
            drm = np.arange(nb)
        else:
            drm = np.zeros((nb, maa[id].shape[0]))
            drm[:, b[id]] = np.eye(nb)
        AM[method][id] = frclim.calcAM([maa[id], baa[id], kaa[id], drm], freq)

    NT[method] = frclim.ntfl(AM[method]["in"], AM[method]["out"], As, freq)

Perform some sanity checks and comparisons against the "CLA" (old fashioned) approach. Again, we see some loss of numerical precision at the lowest frequencies as we noted above for the CLA approach. We'll discuss this below.

In [ ]:
for method in NT:
    print()
    print(method)
    print(f'\t{np.allclose(ifforce["out"][:, pv], NT[method].F[:, pv]) = }')
    print(f'\t{np.allclose(a["out"][:nb, pv], NT[method].A[:, pv]) = }')
    print(f'\t{abs(ifforce["out"] - NT[method].F).max() = }')
    print(f'\t{abs(a["out"][:nb] - NT[method].A).max() = }')

As noted above, we lose some numerical precision in the lowest frequencies. In the NT approach, this appears to be due to very large apparent mass values. To illustrate, here are the apparent mass curves for both approaches for the `x` DOF of the first boundary node for the Source model.

It's plotted on two different scales because it's interesting and informative.

In [ ]:
fig = plt.figure("AM", clear=True, layout="constrained", figsize=(10, 3))
axes = fig.subplots(1, 2, sharey=True)
for method in AM:
    axes[0].semilogy(freq, abs(AM[method]["in"][0, :, 0]), label=method)
    axes[1].loglog(freq, abs(AM[method]["in"][0, :, 0]), label=method)
axes[0].set_xlabel("Frequency (Hz)")
axes[1].set_xlabel("Frequency (Hz)")
axes[0].set_ylabel("X Apparent Mass (mass)");

Similarly, for the Load model:

In [ ]:
fig = plt.figure("AM", clear=True, layout="constrained", figsize=(10, 3))
axes = fig.subplots(1, 2, sharey=True)
for method in AM:
    axes[0].semilogy(freq, abs(AM[method]["out"][0, :, 0]), label=method)
    axes[1].loglog(freq, abs(AM[method]["out"][0, :, 0]), label=method)
axes[0].set_xlabel("Frequency (Hz)")
axes[1].set_xlabel("Frequency (Hz)")
axes[0].set_ylabel("X Apparent Mass (mass)");

One way to think about the apparent mass is that it is the set of forces required to enforce a unit acceleration on one boundary DOF while restraining all other boundary DOF. (That would be one column of the apparent mass matrix at a particular frequency.) In other words, in $f = m a$, if $a = \mathbf{I}$, then the apparent mass $m$ is equal to the force $f$. As frequency approaches zero, the forces required to impose this non-rigid-body boundary condition can become very large, meaning the apparent mass values become very large.

Note that for statically-determinate interfaces, the apparent mass simply approaches the rigid-body mass as frequency approaches 0.0 Hz since the system will move as a rigid-body.

Note also that a similar loss of precision occurs in the CLA approach, so it's not a unique feature of the NT method.

However, even down to 0.01 Hz, the results for both approaches are still quite close, and above 1.0 Hz, all the `np.allclose` checks return True.

For illustration, we'll plot some of the interface forces and boundary accelerations. We can see that NT responses are the same as the CLA responses.

First, compare the interface forces for the 4th boundary node (as an example):

In [ ]:
fig = plt.figure("ifforce", clear=True, layout="constrained", figsize=(8, 11))
frcaxes = fig.subplots(6, 1, sharex=True)

ylabels = ["$F_X$", "$F_Y$", "$F_Z$", "$M_X$", "$M_Y$", "$M_Z$"]
node = 4
for j, ax in enumerate(frcaxes):
    row = j + (node - 1) * 6
    which = "Force" if ylabels[j].startswith("$F") else "Moment"
    for method in NT:
        ax.semilogy(
            freq, abs(NT[method].F[row]), label=f"NT[{method!r}] I/F {which}"
        )
    ax.semilogy(freq, abs(ifforce["out"][row]), "--", label=f"CLA I/F {which}")
    ax.legend()
    ax.set_ylabel(ylabels[j])

frcaxes[0].set_title(f"Magnitude of I/F Force on 'Outboard', Boundary Node #{node}")
frcaxes[-1].set_xlabel("Frequency (Hz)");

Next, compare the interface accelerations for the 4th boundary node. For reference, the free acceleration for this node is also plotted:

In [ ]:
fig = plt.figure("ifacce", clear=True, layout="constrained", figsize=(8, 11))
acceaxes = fig.subplots(6, 1, sharex=True)

ylabels = ["$A_{X}$", "$A_{Y}$", "$A_{Z}$", "$A_{RX}$", "$A_{RY}$", "$A_{RZ}$"]
for j, ax in enumerate(acceaxes):
    row = j + (node - 1) * 6
    for method in NT:
        ax.semilogy(
            freq, abs(NT[method].A[row]), label=f"NT[{method!r}] Coupled Acce"
        )
    ax.semilogy(freq, abs(a["out"][row]), "--", label="CLA Coupled Acce")
    ax.semilogy(freq, abs(As[row]), label="Free Acce")
    ax.legend()
    ax.set_ylabel(ylabels[j])

acceaxes[0].set_title(f"Magnitude of I/F Acceleration, Boundary Node #{node}")
acceaxes[-1].set_xlabel("Frequency (Hz)");